In [4]:
%whos

Variable                         Type                              Data/Info
----------------------------------------------------------------------------
ChatOpenAI                       ModelMetaclass                    <class 'langchain_openai.<...>_models.base.ChatOpenAI'>
Chroma                           ABCMeta                           <class 'langchain_chroma.vectorstores.Chroma'>
Docx2txtLoader                   ABCMeta                           <class 'langchain_communi<...>document.Docx2txtLoader'>
OpenAIEmbeddings                 ModelMetaclass                    <class 'langchain_openai.<...>s.base.OpenAIEmbeddings'>
RecursiveCharacterTextSplitter   ABCMeta                           <class 'langchain_text_sp<...>veCharacterTextSplitter'>
document_list                    list                              n=23
embedding                        OpenAIEmbeddings                  client=<openai.resources.<...>embedding_ctx_length=True
load_dotenv                      function    

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter



# 1. 문서 내용 읽고 분할 =====================================================
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter( #분할된 문서 조각:청크
    chunk_size=1500,
    chunk_overlap=200,

)
document_list=loader.load_and_split(text_splitter=text_splitter) #실질적인 작업은 이 메써드 하나에서 실행 파일 읽고-> 분할-> text_splitter 기준으로 분할

## 2. 임베딩 -> 벡터 데이터베이스에 저장 =====================================================
## 2.1. 환경변수 읽어오기

load_dotenv()

# 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")


# 2.3. 백터 데이터베이스에 저장
# # [방법 1] in memory -> RAM에 저장함.
# database = Chroma.from_documents( #수치화된 분할된 문서를 chroma에 저장
#     documents=document_list, #데이터:문서
#     embedding=embedding,     #수치화

# )


In [ ]:

## [방법2] 로컬에 저장
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma', 
#     collection_name='chroma-law',
# )

# database



In [4]:
%whos

Variable                         Type                              Data/Info
----------------------------------------------------------------------------
ChatOpenAI                       ModelMetaclass                    <class 'langchain_openai.<...>_models.base.ChatOpenAI'>
Chroma                           ABCMeta                           <class 'langchain_chroma.vectorstores.Chroma'>
Docx2txtLoader                   ABCMeta                           <class 'langchain_communi<...>document.Docx2txtLoader'>
OpenAIEmbeddings                 ModelMetaclass                    <class 'langchain_openai.<...>s.base.OpenAIEmbeddings'>
RecursiveCharacterTextSplitter   ABCMeta                           <class 'langchain_text_sp<...>veCharacterTextSplitter'>
database                         Chroma                            <langchain_chroma.vectors<...>ct at 0x0000025812CE2B60>
document_list                    list                              n=23
embedding                        OpenAIEmbedd

In [4]:
## 로컬에 저장된 임베딩 데이터 가져오기
database = Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)

database 

In [5]:
# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 =================================================================
# 3.1. 사용자 질문
# query = '전세사기피해자 금융지원에 대해 설명해주세요.'
query = '전세사기피해자로 인정받기 위한 조건은?'

# 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=3) #앞쪽변수는 매개변수, 뒤쪽변수는 위에 사용자 질문을 저장한값 / 사용자의 정보중 유사한 정보검색을 하는것 -> retrieved docs에 list로 옴.

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ===================================================
# 4.1. 프롬프트 작성
prompt = '''
[identity] 
- 당신은 전세사기피해 법률 전물가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs} 

Question: {query}
'''
## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query
    ) 

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm=ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message


AIMessage(content='전세사기피해자로 인정받기 위한 조건은 다음과 같습니다:\n\n1. **주택의 인도와 주민등록을 완료하고, 확정일자를 갖출 것:** 주택임대차보호법 제3조에 따라 주택의 인도와 주민등록을 마치고, 임대차계약증서상 확정일자를 갖춰야 합니다. 임차권등기를 마친 경우 또는 전세권이 설정된 경우도 포함됩니다.\n\n2. **임차보증금 한도:** 임차인의 임차보증금이 5억원 이하이어야 합니다. 단, 전세사기피해지원위원회에 의해 시ㆍ도별 여건 및 피해자의 여건 등을 고려해 2억원 범위에서 상향 조정될 수 있습니다.\n\n3. **피해 발생 요건:** 임대인의 파산 또는 회생절차 개시, 임차주택의 경매 또는 공매 절차의 개시, 임차인의 집행권원 확보 등으로 인해 2인 이상의 임차인이 임차보증금 반환청구권의 변제를 받지 못하는 피해가 발생했거나 발생할 것이 예상되는 경우.\n\n4. **임대인의 기망 등 사유:** 임대인 또는 관련 인물이 기망, 임차보증금을 반환할 능력이 없는 자에게 임차주택을 양도하거나 임차보증금 반환 능력 없이 다수의 주택을 취득, 임대하는 등 임차보증금 반환채무를 이행하지 않을 의도가 있을 경우.\n\n다만, 다음의 경우는 전세사기피해자로 인정받지 못합니다:\n\n- 임차인이 임차보증금 반환을 위한 보증 또는 보험에 가입하였거나, 임대인이 특정 보증에 가입하여 임차인에게 보증금 전액을 반환할 수 있는 경우.\n- 임차인의 보증금 전액이 주택임대차보호법에 따른 최우선 변제가 가능한 경우.\n- 임차인이 대항력 또는 우선변제권을 행사하여 보증금 전액을 자력으로 회수할 수 있는 경우.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 465, 'prompt_tokens': 3140, 'total_tokens': 3605, 'completion_tokens_details': {'accepted_predicti